In [ ]:
!pip install pymorphy2
!pip install natasha
!pip install -U sentence-transformers

In [ ]:
import numpy as np
from tabulate import tabulate
import pandas as pd
import difflib
import re
from tqdm.autonotebook import tqdm
import copy
import os
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sentence_transformers import SentenceTransformer
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
pd.set_option('display.max_columns', None)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
df1 = pd.read_excel("to_matrix.xlsx")

In [ ]:
topics = df1.columns[2:-1]
result_df = pd.DataFrame()
companies = df1['company'].unique()
results = []
# Обработка данных
for company in companies:
    company_df = df1[df1['company'] == company]
    combined_texts = {topic: '' for topic in topics}

    for topic in topics:
        topic_texts = company_df[company_df['max_topicESG'] == topic]['paragraph'].tolist()
        combined_texts[topic] = ' '.join(topic_texts)

    combined_texts['company'] = company
    results.append(combined_texts)

# Преобразование результатов в DataFrame
result_df = pd.DataFrame(results)

result_df


,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,Персонал в целом,Обучение и развитие,Сотрудники. Вовлеченность и мот,Оплата труда,Сотрудники. Здоровье и благопол,Сотрудники. Наем и увольнение,Сотрудники. Корпоративная культ,Сотрудники. Безопасность и охра,Сотрудники. Профсоюз и Коллекти,Потребители. Доступность,Потребители. Сервис и коммуника,Потребители. Персональные данны,Потребители. Здоровье и благопо,"Потребители. Маркетинг, продажи",Потребители. Удовлетворенность,Потребители. Качество и безопас,Потребители. Ценовая политика,Поставщики в целом,Малый и локальный бизнес,Поставщики. Работники,Поставщики. Экология,Закупки и антикоррупция,Заинтересованные стороны,Коренные народы и местные сообщ,Сотрудники. Волонтерство,Социальные инвестиции и благотв,Отчетность и прозрачность,Отношения с инвесторами,Инновации,Кибербезопасность,Права человека,Лидерство,Риски,Этика и антикоррупция,Корпоративное управление,Устойчивое развитие,company
0,- Пока цветет черемуха - Первая половина мая б...,,,"Помимо этого, на стоимость железный руды оказы...","Похожие сообщения (1): SICE (sice.ru), Москва,...",Также на прошлой неделе Минфин признал несосто...,Природный газ в США +30% за месяц Никель вырос...,"РОССИЙСКАЯ ГАЗЕТА www.rg.ru - Плотина ""Страхов...",- Доход открыт - Истек срок подачи декларации ...,- Дорога с комфортом - О Кабмин разработал кон...,- Что стоит за обсуждением развитии ядерной пр...,- В плановом порядке - Владимир Путин обсудил ...,,"- ""Алиса"" маме не конкурент - Профессор Анатол...",Совкомбанк получил одобрение ЦБ РФ Совкомбанк ...,Альфа-банк провел Alfa Agro Forum - масштабн...,,,- Из-за санкций разработчики могут потерять до...,- Золотое правило - Банки будут следить за пок...,- Совет и защита - Минюст планирует создать во...,- На Марс лететь не надо - Как увеличить продо...,,"- ""Спрогнозировать высокий подъем рек было и м...",- Крымский мост запускали в небо - Стали извес...,"РИА Недвижимость # Новости недвижимости, Москв...",,- Что значит попадание крупных промышленных фи...,,,,,,АКЦИИ Российский рынок Редомициляция Х5 Retail...,,Золото уверенно продолжает рост Трудный выбор ...,"3 М. Дивиденды на каждую акцию - $0,7. Дата вы...","Исследование проводилось по 20 критериям, кот...",- Во главе угля - Производство мяса для шашлык...,- Торг неуместен - Ипотечников предложили осво...,Компания по-прежнему остается лидером в своей ...,- Все на выручку - Что будет с курсом рубля по...,,"Альфа-Банк замещает ""вечный"" евробонд Совет ди...",,"""Альфа-Банк"""
1,Прядильно-ниточный комбинат им. С. М. Кирова -...,,,,,,,,Успешным примером внедрения программы импортоз...,,,,,Георгий Александрович удостоен многочисленных ...,,,,,,,,,"Одним из стратегических активов ""Бронка Групп""...","Совершить порядка 80 % операций с документами,...","Помимо производственных мощностей, холдинг упр...",,,"Опорные активы холдинга Многопрофильность ""Бро...",,,,,Заместитель главы администрации Кировского рай...,,,"Услуги для населения Ключевым проектом ""Бронка...",,Автор: Коробкова Армина Санкт-Петербург - уник...,,,,,,,,"""Бронка Групп"""
2,Столовая школы в Оренбурге после ремонта До ре...,,,Столовая школы в Оренбурге до ремонта После р...,,"Дело ""Фламенко"" Предъявление иска о незаконном...",На челябинские земли АПК выгрузят миллионы тон...,Конкурсный управляющий Сергей Сидоров в разгов...,Дело 2tips Подачу заявки на товарный знак поро...,"В декабре 2023 года ""Черкизово"" и ""Сменка"" под...","С позиции ФАС, рассмотрение заявления о недобр...","АО ""Турбаслинские бройлеры"" (входит в группу ""...",,Арбитражный суд Челябинской области признал не...,Конкурс по определению покупателя состоится в ...,2024-05-18 11:00:00 - 2024-05-18 12:30:00 18 м...,"2.10. Объем (в процентах), в котором исполнено...",,"Суд учел, что система ФГИС ""Меркурий"" закрытая...",,Суд не согласился с такой позицией регулятора ...,Наше учебное заведение начало тесное сотруднич...,Участие в де

In [ ]:
pip install textblob

In [ ]:
from textblob import TextBlob
def analyze_sentiment(text):
    if pd.isna(text) or text == '':
        return 0
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 1
    elif polarity < 0:
        return -1
    else:
        return 0

# Получение списка всех тем (исключая столбец 'company')
topics = result_df.columns[1:]

# Создание новых колонок для сентимента
for topic in topics:
    sentiment_column = topic + '_Окрас'
    result_df[sentiment_column] = result_df[topic].apply(analyze_sentiment)

# Убедимся, что колонка 'company' первая
columns_order = ['company'] + [col for col in result_df.columns if col != 'company']
sentiment = result_df[columns_order]
sentiment

,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,Персонал в целом,Обучение и развитие,Сотрудники. Вовлеченность и мот,Оплата труда,Сотрудники. Здоровье и благопол,Сотрудники. Наем и увольнение,Сотрудники. Корпоративная культ,Сотрудники. Безопасность и охра,Сотрудники. Профсоюз и Коллекти,Потребители. Доступность,Потребители. Сервис и коммуника,Потребители. Персональные данны,Потребители. Здоровье и благопо,"Потребители. Маркетинг, продажи",Потребители. Удовлетворенность,Потребители. Качество и безопас,Потребители. Ценовая политика,Поставщики в целом,Малый и локальный бизнес,Поставщики. Работники,Поставщики. Экология,Закупки и антикоррупция,Заинтересованные стороны,Коренные народы и местные сообщ,Сотрудники. Волонтерство,Социальные инвестиции и благотв,Отчетность и прозрачность,Отношения с инвесторами,Инновации,Кибербезопасность,Права человека,Лидерство,Риски,Этика и антикоррупция,Корпоративное управление,Устойчивое развитие,Климат_Окрас,Энергия_Окрас,Воздух_Окрас,Вода_Окрас,Отходы и циклическая экономика_Окрас,Биоразнообразие_Окрас,Рекультивация земель_Окрас,Экологичность продукта_Окрас,Персонал в целом_Окрас,Обучение и развитие_Окрас,Сотрудники. Вовлеченность и мот_Окрас,Оплата труда_Окрас,Сотрудники. Здоровье и благопол_Окрас,Сотрудники. Наем и увольнение_Окрас,Сотрудники. Корпоративная культ_Окрас,Сотрудники. Безопасность и охра_Окрас,Сотрудники. Профсоюз и Коллекти_Окрас,Потребители. Доступность_Окрас,Потребители. Сервис и коммуника_Окрас,Потребители. Персональные данны_Окрас,Потребители. Здоровье и благопо_Окрас,"Потребители. Маркетинг, продажи_Окрас",Потребители. Удовлетворенность_Окрас,Потребители. Качество и безопас_Окрас,Потребители. Ценовая политика_Окрас,Поставщики в целом_Окрас,Малый и локальный бизнес_Окрас,Поставщики. Работники_Окрас,Поставщики. Экология_Окрас,Закупки и антикоррупция_Окрас,Заинтересованные стороны_Окрас,Коренные народы и местные сообщ_Окрас,Сотрудники. Волонтерство_Окрас,Социальные инвестиции и благотв_Окрас,Отчетность и прозрачность_Окрас,Отношения с инвесторами_Окрас,Инновации_Окрас,Кибербезопасность_Окрас,Права человека_Окрас,Лидерство_Окрас,Риски_Окрас,Этика и антикоррупция_Окрас,Корпоративное управление_Окрас,Устойчивое развитие_Окрас,company_Окрас
0,"""Альфа-Банк""",- Пока цветет черемуха - Первая половина мая б...,,,"Помимо этого, на стоимость железный руды оказы...","Похожие сообщения (1): SICE (sice.ru), Москва,...",Также на прошлой неделе Минфин признал несосто...,Природный газ в США +30% за месяц Никель вырос...,"РОССИЙСКАЯ ГАЗЕТА www.rg.ru - Плотина ""Страхов...",- Доход открыт - Истек срок подачи декларации ...,- Дорога с комфортом - О Кабмин разработал кон...,- Что стоит за обсуждением развитии ядерной пр...,- В плановом порядке - Владимир Путин обсудил ...,,"- ""Алиса"" маме не конкурент - Профессор Анатол...",Совкомбанк получил одобрение ЦБ РФ Совкомбанк ...,Альфа-банк провел Alfa Agro Forum - масштабн...,,,- Из-за санкций разработчики могут потерять до...,- Золотое правило - Банки будут следить за пок...,- Совет и защита - Минюст планирует создать во...,- На Марс лететь не надо - Как увеличить продо...,,"- ""Спрогнозировать высокий подъем рек было и м...",- Крымский мост запускали в небо - Стали извес...,"РИА Недвижимость # Новости недвижимости, Москв...",,- Что значит попадание крупных промышленных фи...,,,,,,АКЦИИ Российский рынок Редомициляция Х5 Retail...,,Золото уверенно продолжает рост Трудный выбор ...,"3 М. Дивиденды на каждую акцию - $0,7. Дата вы...","Исследование проводилось по 20 критериям, кот...",- Во главе угля - Производство мяса для шашлык...,- Торг неуместен - Ипотечников предложили осво...,Компания по-прежнему остается лидером в своей ...,- Все на выручку - Что будет с курсом рубля по...,,"Альфа-Банк замещает ""вечный"" евробонд Совет ди...",,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,0
1,"""Бронка Групп""",Прядильно-ниточ

In [ ]:
def count_topic_occurrences(group):
    return group['max_topicESG'].value_counts()

# Группируем по компаниям и применяем функцию count_topic_occurrences
df2 = df_cos.assign(company=df_cos["company"].str.split('; ')).explode("company").sort_values(by=["company"], ascending=[True]).reset_index(drop=True)
result = df2.groupby('company').apply(count_topic_occurrences).unstack(fill_value=0)

# Переименовываем колонки для удобства
#result.columns.name = None
result = result.reset_index()

combined_df = pd.concat([result, sentiment.iloc[:, 46:-1]], axis=1)
combined_df

,company,0,Биоразнообразие,Вода,Воздух,Заинтересованные стороны,Закупки и антикоррупция,Инновации,Кибербезопасность,Климат,Коренные народы и местные сообщ,Корпоративное управление,Лидерство,Малый и локальный бизнес,Обучение и развитие,Оплата труда,Отношения с инвесторами,Отходы и циклическая экономика,Отчетность и прозрачность,Персонал в целом,Поставщики в целом,Поставщики. Работники,Поставщики. Экология,Потребители. Доступность,Потребители. Здоровье и благопо,Потребители. Качество и безопас,"Потребители. Маркетинг, продажи",Потребители. Персональные данны,Потребители. Сервис и коммуника,Потребители. Удовлетворенность,Потребители. Ценовая политика,Права человека,Рекультивация земель,Риски,Сотрудники. Безопасность и охра,Сотрудники. Вовлеченность и мот,Сотрудники. Волонтерство,Сотрудники. Здоровье и благопол,Сотрудники. Корпоративная культ,Сотрудники. Наем и увольнение,Сотрудники. Профсоюз и Коллекти,Социальные инвестиции и благотв,Устойчивое развитие,Экологичность продукта,Экология в целом,Энергия,Этика и антикоррупция,Климат_Окрас,Энергия_Окрас,Воздух_Окрас,Вода_Окрас,Отходы и циклическая экономика_Окрас,Биоразнообразие_Окрас,Рекультивация земель_Окрас,Экологичность продукта_Окрас,Персонал в целом_Окрас,Обучение и развитие_Окрас,Сотрудники. Вовлеченность и мот_Окрас,Оплата труда_Окрас,Сотрудники. Здоровье и благопол_Окрас,Сотрудники. Наем и увольнение_Окрас,Сотрудники. Корпоративная культ_Окрас,Сотрудники. Безопасность и охра_Окрас,Сотрудники. Профсоюз и Коллекти_Окрас,Потребители. Доступность_Окрас,Потребители. Сервис и коммуника_Окрас,Потребители. Персональные данны_Окрас,Потребители. Здоровье и благопо_Окрас,"Потребители. Маркетинг, продажи_Окрас",Потребители. Удовлетворенность_Окрас,Потребители. Качество и безопас_Окрас,Потребители. Ценовая политика_Окрас,Поставщики в целом_Окрас,Малый и локальный бизнес_Окрас,Поставщики. Работники_Окрас,Поставщики. Экология_Окрас,Закупки и антикоррупция_Окрас,Заинтересованные стороны_Окрас,Коренные народы и местные сообщ_Окрас,Сотрудники. Волонтерство_Окрас,Социальные инвестиции и благотв_Окрас,Отчетность и прозрачность_Окрас,Отношения с инвесторами_Окрас,Инновации_Окрас,Кибербезопасность_Окрас,Права человека_Окрас,Лидерство_Окрас,Риски_Окрас,Этика и антикоррупция_Окрас,Корпоративное управление_Окрас,Устойчивое развитие_Окрас
0,"""Альфа-Банк""",4,1,5,1,0,0,9,13,0,0,1,1,5,4,0,44,1,3,17,0,0,0,4,2,1,0,2,7,5,8,3,1,5,0,8,2,4,5,2,0,0,0,2,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0
1,"""Бронка Групп""",0,0,0,0,0,0,1,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"""Черкизово""",11,1,0,1,0,5,5,7,0,0,1,1,32,11,0,10,6,12,27,1,0,0,1,2,4,3,2,0,9,3,16,4,1,1,14,1,4,3,4,0,2,2,26,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cosmos Hotel Group,4,3,4,0,0,3,23,2,0,0,9,1,8,4,0,11,0,2,46,0,0,0,7,2,0,4,0,2,3,5,9,2,4,0,7,1,9,2,3,1,4,2,5,99,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
4,En+ Group,0,1,1,0,0,0,2,1,0,0,1,1,3,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,3,0,2,0,1,0,1,0,0,0,0,0,0,0,1,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Тинькофф Банк,5,0,0,0,0,0,2,2,0,0,0,0,1,0,0,14,0,1,1,0,0,0,0,0,1,0,0,0,1,3,0,0,4,0,3,0,0,2,0,0,0,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
122,УКООЛО,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,

In [ ]:
companies = pd.read_excel("/content/companies/RAEX2024 (2).xlsx")
main_dataset = combined_df.reset_index()

def clean_company_name(name):
    if "Segezha Group" in name:
        return "Сегежа Групп".strip().lower()
    if "Билайн (Вымпелком СЗФО)" in name:
        return "Вымпелком".strip().lower()
    if "ПИК" in name:
        return "Группа Компаний ПИК".strip().lower()
    name = str(name)
    name = re.sub(r'\bПАО\b', '', name)
    name = re.sub(r'\bГруппа\b', '', name)
    name = re.sub(r'\bБанк\b', '', name)
    name = re.sub(r',.*$', '', name)
    name = re.sub(r'\s*\(.*?\)\s*', '', name)
    return str(name).replace('«', '').replace('»', '').replace('"', '').replace('”', '').replace('“', '').strip().lower()


main_dataset['clean_company'] = main_dataset['company'].apply(clean_company_name)

# Очистим названия компаний в Excel файле
companies['clean_company'] = companies['Название'].apply(clean_company_name)

# Шаг 3: Присоединить данные из Excel файла к основному датасету
merged_data = main_dataset.merge(companies[['clean_company', 'ESG-рейтинг', 'E-рейтинг', 'S-рейтинг', 'G-рейтинг']],
                                 how='left',
                                 on='clean_company')

# Удалим лишние колонки
merged_data.drop(columns=['clean_company'], axis=1, inplace=True)

# Вернем индексы на место
#merged_data.set_index('company', inplace=True)
merged_data = merged_data.drop(['index'], axis = 1)
merged_data

,company,0,Биоразнообразие,Вода,Воздух,Заинтересованные стороны,Закупки и антикоррупция,Инновации,Кибербезопасность,Климат,Коренные народы и местные сообщ,Корпоративное управление,Лидерство,Малый и локальный бизнес,Обучение и развитие,Оплата труда,Отношения с инвесторами,Отходы и циклическая экономика,Отчетность и прозрачность,Персонал в целом,Поставщики в целом,Поставщики. Работники,Поставщики. Экология,Потребители. Доступность,Потребители. Здоровье и благопо,Потребители. Качество и безопас,"Потребители. Маркетинг, продажи",Потребители. Персональные данны,Потребители. Сервис и коммуника,Потребители. Удовлетворенность,Потребители. Ценовая политика,Права человека,Рекультивация земель,Риски,Сотрудники. Безопасность и охра,Сотрудники. Вовлеченность и мот,Сотрудники. Волонтерство,Сотрудники. Здоровье и благопол,Сотрудники. Корпоративная культ,Сотрудники. Наем и увольнение,Сотрудники. Профсоюз и Коллекти,Социальные инвестиции и благотв,Устойчивое развитие,Экологичность продукта,Экология в целом,Энергия,Этика и антикоррупция,Климат_Окрас,Энергия_Окрас,Воздух_Окрас,Вода_Окрас,Отходы и циклическая экономика_Окрас,Биоразнообразие_Окрас,Рекультивация земель_Окрас,Экологичность продукта_Окрас,Персонал в целом_Окрас,Обучение и развитие_Окрас,Сотрудники. Вовлеченность и мот_Окрас,Оплата труда_Окрас,Сотрудники. Здоровье и благопол_Окрас,Сотрудники. Наем и увольнение_Окрас,Сотрудники. Корпоративная культ_Окрас,Сотрудники. Безопасность и охра_Окрас,Сотрудники. Профсоюз и Коллекти_Окрас,Потребители. Доступность_Окрас,Потребители. Сервис и коммуника_Окрас,Потребители. Персональные данны_Окрас,Потребители. Здоровье и благопо_Окрас,"Потребители. Маркетинг, продажи_Окрас",Потребители. Удовлетворенность_Окрас,Потребители. Качество и безопас_Окрас,Потребители. Ценовая политика_Окрас,Поставщики в целом_Окрас,Малый и локальный бизнес_Окрас,Поставщики. Работники_Окрас,Поставщики. Экология_Окрас,Закупки и антикоррупция_Окрас,Заинтересованные стороны_Окрас,Коренные народы и местные сообщ_Окрас,Сотрудники. Волонтерство_Окрас,Социальные инвестиции и благотв_Окрас,Отчетность и прозрачность_Окрас,Отношения с инвесторами_Окрас,Инновации_Окрас,Кибербезопасность_Окрас,Права человека_Окрас,Лидерство_Окрас,Риски_Окрас,Этика и антикоррупция_Окрас,Корпоративное управление_Окрас,Устойчивое развитие_Окрас,ESG-рейтинг,E-рейтинг,S-рейтинг,G-рейтинг
0,"""Альфа-Банк""",4,1,5,1,0,0,9,13,0,0,1,1,5,4,0,44,1,3,17,0,0,0,4,2,1,0,2,7,5,8,3,1,5,0,8,2,4,5,2,0,0,0,2,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,0,0,0,0,0,0,CC,C,C,CCC
1,"""Бронка Групп""",0,0,0,0,0,0,1,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,"""Черкизово""",11,1,0,1,0,5,5,7,0,0,1,1,32,11,0,10,6,12,27,1,0,0,1,2,4,3,2,0,9,3,16,4,1,1,14,1,4,3,4,0,2,2,26,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CC,C,CC,B
3,Cosmos Hotel Group,4,3,4,0,0,3,23,2,0,0,9,1,8,4,0,11,0,2,46,0,0,0,7,2,0,4,0,2,3,5,9,2,4,0,7,1,9,2,3,1,4,2,5,99,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN
4,En+ Group,0,1,1,0,0,0,2,1,0,0,1,1,3,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,3,0,2,0,1,0,1,0,0,0,0,0,0,0,1,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BBB,BB,BBB,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Тинькофф Банк,5,0,0,0,0,0,2,2,0,0,0,0,1,0,0,14,0,1,1,0,0,0,0,0,1,0,0,0,1,3,0,0,4,0,3,0,0,2,0,0,0,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [ ]:
merged_data.to_excel("matrix.xlsx", index=False)